In [26]:
''' This code generates an autoregressive time series with added normal noise given its order and initial values 
    and stores the data in the Data.xlsx file. 
    specify the row to store the data in variable r
'''
# set row in Data.xlsx file
r = 3
# set standard deviation
sigma = 0.1
# set path of file
path = "Data.xlsx"

import numpy as np
import pandas as pd
import openpyxl
import copy
# Get which order the user wants for the autoregressive model
order = int(input("Order for Autoregressive model"))

# create an array of shape(order, ) and input coefficients in it
coeff = np.zeros((order, ))
for idx in range(order) :
    coeff[idx] = float(input("coefficient : " + str(idx + 1)))
coeff.reshape((-1, 1))
const = float(input("coefficiet : "+ str(order+1)))
# print(coeff)

# get the initial values of the time series
initial = np.zeros((order, ))
for idx in range(order):
    initial[idx] = float(input("initial value at time " + str(idx + 1) + " : "))
# print (initial)

# get number of time steps
length = int(input("Length of time series to be generated : "))

# write initial data in the excel file
wb = openpyxl.load_workbook(path)
ws = wb.active

for idx in range(order):
    ws.cell(row=r, column=idx+1).value = initial[idx]

# generate the autoregressive time series of the given order
# latest stores the latest values of the times series
latest = copy.deepcopy(initial)

for idx in range(length):
    # multiply the matrices
    val = np.matmul(latest, coeff) + const + np.random.normal(0, sigma)
    print(latest, val)
    for i in range(latest.size - 1):
        latest[i] = latest[i + 1]
    latest[order-1] = val
    ws.cell(row=r, column=order+idx+1).value = val

wb.save(path)

[ 1. 13. 15.] 38.51838176104117
[13.         15.         38.51838176] 59.086225496471954
[15.         38.51838176 59.0862255 ] 89.29968209043334
[38.51838176 59.0862255  89.29968209] 135.11613110115323
[ 59.0862255   89.29968209 135.1161311 ] 194.51642753010614
[ 89.29968209 135.1161311  194.51642753] 279.13538964582756
[135.1161311  194.51642753 279.13538965] 396.1949949367495
[194.51642753 279.13538965 396.19499494] 558.1099061899354
[279.13538965 396.19499494 558.10990619] 783.3671427738657
[396.19499494 558.10990619 783.36714277] 1095.8678223748968


In [27]:
''' This code is used for retreiving the parameters of an AR process
'''
def prediction(coefficients, laggedTargets):
    predictionValue = coefficients[0]
    for i in range(laggedTargets.size):
        predictionValue = coefficients[i + 1] * laggedTargets[i]
    return predictionValue

def findCoefficients(data, k):
    size = data.size
    data_old = np.array([]).reshape(size - k, 0)
    data_new = data[k:size]
    k -= 1
    size -= 1
    while(k >= 0):
        data_old = np.c_[data_old, data[k:size]]
        k -= 1
        size -= 1
    
    data_old = np.c_[data_old, np.ones(size - k)]
    Y = data_new.reshape(size - k, 1)
    A = data_old
    print("------A matrix------")
    print(A)
    print("------Y matrix------")
    print(Y)
    theta = np.matmul(np.matmul(np.linalg.pinv(np.matmul(A.T, A)), A.T), Y)
    # theta = np.dot(np.linalg.pinv(A), Y)
    print("------Theta matrix------")
    print(theta)

    coefficients = theta.reshape(data.size - size)
    return coefficients

In [28]:
''' This code is used to read AR data from Data.xlsx file and predict the coefficients
'''

wb = openpyxl.load_workbook(path)
wa = wb.active
tot_cols = length + initial.size

# try printing
y = np.zeros(tot_cols)
for idx in range(1, tot_cols+1):
    y[idx-1] = wa.cell(row = r, column = idx).value

findCoefficients(y, order)
print("-----original Theta matrix-----")
print(coeff)
print(const)

------A matrix------
[[ 15.          13.           1.           1.        ]
 [ 38.51838176  15.          13.           1.        ]
 [ 59.0862255   38.51838176  15.           1.        ]
 [ 89.29968209  59.0862255   38.51838176   1.        ]
 [135.1161311   89.29968209  59.0862255    1.        ]
 [194.51642753 135.1161311   89.29968209   1.        ]
 [279.13538965 194.51642753 135.1161311    1.        ]
 [396.19499494 279.13538965 194.51642753   1.        ]
 [558.10990619 396.19499494 279.13538965   1.        ]
 [783.36714277 558.10990619 396.19499494   1.        ]]
------Y matrix------
[[  38.51838176]
 [  59.0862255 ]
 [  89.29968209]
 [ 135.1161311 ]
 [ 194.51642753]
 [ 279.13538965]
 [ 396.19499494]
 [ 558.10990619]
 [ 783.36714277]
 [1095.86782237]]
------Theta matrix------
[[ 0.49906319]
 [ 0.79875349]
 [ 0.60326082]
 [20.07604642]]
-----original Theta matrix-----
[0.6 0.8 0.5]
20.0
